In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.font_manager import FontProperties

import csv
import numpy as np
import os

In [ ]:
import sys

default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.setdefaultencoding("utf-8")
sys.stdout = default_stdout
sys.stderr = default_stderr


In [ ]:
!pip2 show matplotlib

## Some defaults for generated plots

In [ ]:
mpl.rc('legend', fontsize='large')
mpl.rc('axes', labelsize='large')
mpl.rc('figure', figsize=(6,4), autolayout=True)

## Just ensuring the output directories exists...

In [ ]:
output_dir = 'output'
output_subdirs = ['ratio_base', 'multiple_runs', 'price_factor_base']

for sub in output_subdirs:
    try:
        os.makedirs(os.path.join(output_dir, sub))
    except OSError:
        pass

# Generation of plots for my internship repport

Using the data of previous executions of DCWoRMS, we try to parse the stats files and to display *beautiful* charts with Matplotlib.
To make the process as much automated as possible, we define several helpers, which are used for some kind of data...

## List of files and experiment names

In [ ]:
source_ratio_impact = [
    ('wsum', 'stats_sum.dat'),
    ('wsinhsum', 'stats_sinh-ext.dat'),
    ('fuzzy-elec', 'stats_fuzzy-electrical.dat'),
    ('fuzzy-it', 'stats_fuzzy-it.dat')
]

source_ratio_impact_path = 'data/ratio_base'

## Centralized plot and bar style control
Style information are centralized thereafter (line type, marker, color...) for the different kind of chart needed.
The goal is to 1) keep the same style for the same dataset everywhere and 2) have a single place where to customize styles if needed (e.g. to output a black-only printable figure, or to use it elsewhere by adapting the style).

### Styles for ploting metrics values

In [ ]:
plot_metrics_stats = {
    'qos' : ('blue', '^', '-', 'SLA violation rate'),
    'cost' : ('red', 'o', '-', 'Cost of grid energy'),
    'nrpart' : ('green', 's', '-', 'Non-renewable energy part'),
    'grid' : ('purple', 'D', '-', 'Energy consumption from grid'),
    'avgprice' : ('red', 'v', '-', 'Average price of grid energy'),
}

### Styles for comparing different approaches/variants on a given metric

In [ ]:
plot_compare_approaches = {
    'wsum' : ('blue', '>', '-', 'Weighted sum (A)'),
    'wsinhsum' : ('darkblue', '<', '-', 'Weighted sinh sum (A)'),
    'fuzzy-it' : ('purple', '^', '-', 'Fuzzy-based IT (A)'),
    'price-wsum' : ('red', '*', '-', 'Weighted sum (B)'),
    'price-wsinhsum' : ('firebrick', 'x', '-', 'Weighted sinh sum (B)'),
    'price-fuzzy-it' : ('orangered', '+', '-', 'Fuzzy-based IT (B)'),
    'greenslot-original' : ('lime', 's', '--', 'GreenSlot (original)'),
    'greenslot-improved' : ('limegreen', '<', '-', 'GreenSlot (part modif)'),
    'greenslot-full' : ('green', 'D', '-', 'GreenSlot (modified)'),
}

### Styles for bar plot comparing different approaches

In [ ]:
bar_compare_approaches = {
    'wsum' : ('#3182bd', '/', 'Somme pondérée (A)'),
    'wsinhsum' : ('#deebf7', '', 'Weighted sinh sum (A)'),
    'fuzzy-it' : ('#9ecae1', '//', 'Fuzzy-based (A)'),
    'price-wsum' : ('#6a51a3', '/', 'ABBSH, somme pondérée'),
    'price-fuzzy-it' : ('#9e9ac8', '//', 'ABBSH, fuzzy-based'),
    'greenslot-original' : ('#a1d99b', '\\', 'GreenSlot (original)'),
    'greenslot-full' : ('#31a354', '\\\\', 'GreenSlot (modifié)'),
    'firstfit-nooff' : ('#de2d26', '-', 'First-fit (no poweroff)'),
    'firstfit' : ('#fc9272', 'x', 'First-fit')
}

### Some helpers to retrieve plot style attributes...

In [ ]:
def getpstyle(style, name):
    return {
        'color': style[name][0],
        'marker': style[name][1],
        'linestyle': style[name][2],
        'label': style[name][3]
    }

In [ ]:
def getbstyle(style, name):
    return {
        'color': style[name][0],
        'hatch': style[name][1],
        'edgecolor': 'black',
        'label': style[name][2]
    }

## Utility functions
For reading data from stats files and creating some kind of charts.

In [ ]:
def read_data(filename, fields):
    dumbret = {}
    for f in fields:
        dumbret[f] = []
    
    with open(filename, 'rb') as csvfile:
        testreader = csv.reader(csvfile, delimiter='\t', quotechar='"')
        for row in testreader:
            if not row[0].startswith('#'):
                #print ', '.join(row)
                
                for f in fields:
                    dumbret[f].append(float(row[f]))
    listret = []
    for f in fields:
        listret.append(dumbret[f])
    
    return tuple(listret)

In [ ]:
def draw_chart_1(stats, name, title = None, xlabel=r"$\alpha$ parameter", legend=False):
    fig, ax1 = plt.subplots()
    
    global plot_metrics_stats
    stylef = lambda n: getpstyle(plot_metrics_stats, n)
    
    if title is not None:
        ax1.set_title(title)
    ax1.set_ylabel('Ratio (%)')
    ax1.set_xlabel(xlabel)

    ax1.plot(stats['ratio'], [x * 100.0 for x in stats['grid']], **stylef('nrpart'))
    ax1.plot(stats['ratio'], [x * 100.0 for x in stats['missed']], **stylef('qos'))
    
    if legend:
        ax1.legend(loc="upper center")
    
    ax1.set_axisbelow(True)
    ax1.grid(zorder=0, axis='y')
    
    fig.savefig("output/ratio_base/{}.pdf".format(name))

## Stats related to the impact of the parameters in objective functions

In [ ]:
stats_ratio_impact = {}

for (sname, sdir) in source_ratio_impact:
    spath = os.path.join(source_ratio_impact_path, sdir)
    #   (flex, grid, price, missed) = read_data(spath, [0, 3, 5, 2])
    #   stats_ratio_impact[sname] = {'flex' : flex, 'grid' : grid, 'price': price, 'missed': missed}
    values = read_data(spath, [0, 1, 4, 5, 6, 3])
    zipped = zip(*values)
    
    # re-structure the data...
    #zip(price_factor, wl_instance, grid, energy, price, missed)
    result = []
    for pf in sorted(set(values[0])):
        good = [item for item in zipped if item[0] == pf]
        result.append([np.average(item) for item in zip(*good)])
    
    (ratio, _, grid, energy, price, missed) = zip(*result)
    
    stats_ratio_impact[sname] = {'ratio' : ratio, 'energy' : energy, 'grid' : grid, 'price': price, 'missed': missed}

In [ ]:
draw_chart_1(stats_ratio_impact['wsum'], 'wsum', legend=True)#, r"Impact de $\alpha$ pour la somme pondérée simple")

In [ ]:
draw_chart_1(stats_ratio_impact['wsinhsum'], 'wsinhsum')#, r"Impact de $\alpha$ pour la somme pondérée des sinus hyperboliques")

In [ ]:
draw_chart_1(stats_ratio_impact['fuzzy-it'], 'fuzzy-it')#, r"Impact de $\alpha$ pour le fuzzy-it")

In [ ]:
draw_chart_1(stats_ratio_impact['fuzzy-elec'], 'fuzzy-elec')#, r"Impact de $\alpha$ pour le fuzzy-elec")

# Grid price variant behavior depending on price factor
Price factor is used to define the impact of grid price on electrical attractivity

In [ ]:
source_pricef_impact = [
    ('wsum', 'stats_wsum.dat'),
    ('wsinhsum', 'stats_wsinhsum.dat'),
    #('wsinhsum', 'data/ratio_base/stats_sinh-ext.dat'),
    #('fuzzy-elec', 'data/ratio_base/stats_fuzzy-electrical.dat'),
    ('fuzzy-it', 'stats_fuzzy-it.dat')
]
source_pricef_impact_base = 'data/price_factor_base/'

In [ ]:
def draw_chart_pricef_3(stats, name, title = None, xlabel=r"$\lambda$ (price factor)", legend=False):
    fig, ax1 = plt.subplots()
    
    global plot_metrics_stats
    stylef = lambda n: getpstyle(plot_metrics_stats, n)
    
    if title is not None:
        ax1.set_title(title)
    #ax1.set_ylabel('Ratio')
    ax1.set_ylabel('Power consumed from grid (kWh)')
    ax1.set_xlabel(xlabel)

    gridpower = [gpart * energy for (gpart, energy) in zip(stats['grid'], stats['energy'])]
    lns1 = ax1.plot(stats['price_factor'], gridpower, **stylef('grid'))
    #lns1 = ax1.plot(stats['price_factor'], stats['grid'],  **stylef('nrpart'))
    #lns2 = ax1.plot(stats['price_factor'], stats['missed'],  **stylef('qos'))
    
    ax2 = ax1.twinx()
    ax2.set_ylabel('Grid electricity cost ($)')
    lns3 = ax2.plot(stats['price_factor'], stats['price'],  **stylef('cost'))
    
    # tricks for both axis legends at same place...
    #lns = lns1 + lns2 + lns3
    lns = lns1 + lns3
    labels = [l.get_label() for l in lns]
    
    ax1.set_axisbelow(True)
    ax1.grid(zorder=0, axis='y')
    
    if legend:
        ax1.legend(lns, labels, loc="upper center")
    
    fig.savefig("output/price_factor_base/both_{}.pdf".format(name))

In [ ]:
def draw_chart_pricef_4(stats, name, title = None, xlabel=r"$\lambda$ (price factor)"):
    fig, ax1 = plt.subplots()
    
    global plot_metrics_stats
    stylef = lambda n: getpstyle(plot_metrics_stats, n)
    
    if title is not None:
        ax1.set_title(title)
    ax1.set_ylabel('Power consumed from grid (kWh)')
    ax1.set_xlabel(xlabel)
    
    gridpower = [gpart * energy for (gpart, energy) in zip(stats['grid'], stats['energy'])]
    avgprice = [cost / gpower for (gpower, cost) in zip(gridpower, stats['price'])]

    lns1 = ax1.plot(stats['price_factor'], gridpower, **stylef('grid'))
    #lns2 = ax1.plot(stats['price_factor'], stats['missed'],  **stylef('qos'))
    
    ax2 = ax1.twinx()
    #ax2.set_ylabel('Grid electricity cost ($)')
    #lns3 = ax2.plot(stats['price_factor'], stats['price'],  **stylef('cost'))
    ax2.set_ylabel('Energy price ($/kWh)')
    lns3 = ax2.plot(stats['price_factor'], avgprice,  **stylef('avgprice'))
    
    # tricks for both axis legends at same place...
    lns = lns1 + lns3
    labels = [l.get_label() for l in lns]
    ax1.legend(lns, labels, loc="right")
    
    ax1.set_axisbelow(True)
    ax1.grid(zorder=0, axis='y')
    
    #fig.savefig("output/price_factor_base/both_{}.pdf".format(name))

## Temporary tests : Elec variant B, price factor impact with multiple runs

In [ ]:
stats_pricef_impact_multi = {}

for (sname, sdir) in source_pricef_impact:
    spath = os.path.join(source_pricef_impact_base, sdir)
    #   (flex, grid, price, missed) = read_data(spath, [0, 3, 5, 2])
    #   stats_ratio_impact[sname] = {'flex' : flex, 'grid' : grid, 'price': price, 'missed': missed}
    values = read_data(spath, [0, 1, 4, 5, 6, 3])
    zipped = zip(*values)
    
    # re-structure the data...
    #zip(price_factor, wl_instance, grid, energy, price, missed)
    result = []
    for pf in sorted(set(values[0])):
        good = [item for item in zipped if item[0] == pf]
        result.append([np.average(item) for item in zip(*good)])
    
    (price_factor, wl_instance, grid, energy, price, missed) = zip(*result)
    
    stats_pricef_impact_multi[sname] = {'price_factor' : price_factor, 'energy' : energy, 'grid' : grid, 'price': price, 'missed': missed}

In [ ]:
draw_chart_pricef_4(stats_pricef_impact_multi['wsum'], 'wsum')
draw_chart_pricef_3(stats_pricef_impact_multi['wsum'], 'wsum', legend=True)

In [ ]:
draw_chart_pricef_4(stats_pricef_impact_multi['fuzzy-it'], 'fuzzy-it')
draw_chart_pricef_3(stats_pricef_impact_multi['fuzzy-it'], 'fuzzy-it')

In [ ]:
draw_chart_pricef_4(stats_pricef_impact_multi['wsinhsum'], 'wsinhsum')
draw_chart_pricef_3(stats_pricef_impact_multi['wsinhsum'], 'wsinhsum')

## Algorithm behavior in funtion of flexibility factor
Using the previous result, we select some of the "best" values for each algorithm, and compare them with GreenSlot variants.

### Comparison of the approaches

In [ ]:
def draw_chart_compare_1(stats_func, name, items, styles = None, title = None, ybounds=(None, None),
                         legendloc = "upper right", xlabel=r"Workload flexibility factor", ylabel=r"Ratio"):
    fig, ax1 = plt.subplots()
    
    if styles is None:
        global plot_compare_approaches
        styles = plot_compare_approaches
    
    stylef = lambda n: getpstyle(styles, n)
    
    if title is not None:
        ax1.set_title(title)
    ax1.set_ylabel(ylabel)
    ax1.set_xlabel(xlabel)

    for item in items:
        (xval, yval) = stats_func(item)
        ax1.plot(xval, yval, **stylef(item))
    
    if legendloc is not None:
        ax1.legend(loc=legendloc)
    ax1.set_axisbelow(True)
    ax1.grid(zorder=0, axis='y')
    
    ax1.set_ybound(lower=ybounds[0], upper=ybounds[1])
    
    fig.savefig("output/flex_base/{}.pdf".format(name))

In [ ]:
def get_nonrenew(algo):
    global stats_flex_impact
    stats = stats_flex_impact[algo]
    valx = stats['flex']
    valy = []
    for i in range(len(stats['grid'])):
        valy.append(stats['grid'][i] * stats['energy'][i])
    return (valx, valy)

def get_price(algo):
    global stats_flex_impact
    stats = stats_flex_impact[algo]
    return (stats['flex'], stats['price'])

def get_fail_rate(algo, exclude=None):
    global stats_flex_impact
    stats = stats_flex_impact[algo]
    
    valx = []
    valy = []
    for i in range(len(stats['flex'])):
        v = stats['flex'][i]
        if exclude is None or v not in exclude:
            valx.append(v)
            valy.append(stats['missed'][i] * 100.0)
            
    #return (stats['flex'], [x * 100.0 for x in stats['missed']])
    return (valx, valy)

In [ ]:
algos_set_1 = ['wsum', 'fuzzy-it', 'price-wsum', 'price-fuzzy-it', 'greenslot-full', 'greenslot-original']
algos_set_2 = ['wsum', 'fuzzy-it', 'price-wsum', 'price-fuzzy-it', 'greenslot-full']

#algos_set_final = ['wsum', 'fuzzy-it', 'price-wsum', 'price-fuzzy-it', 'greenslot-full', 'greenslot-original', 'firstfit']
# For Compas...
algos_set_final = ['price-wsum', 'price-fuzzy-it', 'greenslot-full', 'firstfit']
algos_set_final_noff = ['wsum', 'fuzzy-it', 'price-wsum', 'price-fuzzy-it', 'greenslot-full', 'greenslot-original']

algos_set_gs = ['greenslot-original', 'greenslot-improved', 'greenslot-full']
algos_set_noprice = ['greenslot-full', 'wsum', 'fuzzy-it', 'wsinhsum']
algos_set_price = ['greenslot-full', 'price-wsum', 'price-fuzzy-it', 'price-wsinhsum']


## Statistics for multiple (randomly generated) workloads
Better statistics are provided by repeating the experiments using slightly different workloads.
Each is generated with the same parameters, but using a different random seed.

We can use these stats to evaluate the average performances of the various algorihms.

In [ ]:
source_multiple_runs_base = [
    ('price-wsum', 'price_sum (0.55)'),
    ('price-wsinhsum', 'price_sinhsum (0.55)'),
    ('price-fuzzy-it', 'price_fuzzy-it (0.75)'),
    ('wsum', 'sum (0.55)'),
    ('wsinhsum', 'sinhsum (0.55)'),
    ('fuzzy-it', 'fuzzy-it (0.75)'),
    ('firstfit', 'firstfit'),
    ('firstfit-nooff', 'firstfit-no-poweroff'),
    ('greenslot-original', 'greenslot-original'),
    ('greenslot-improved', 'greenslot-improved'),
    ('greenslot-full', 'greenslot-full')
]
source_multiple_runs_base_path = 'data/multiple_runs/stats_{}.dat'

In [ ]:
stats_multiple_runs = {}

for (sname, sdir) in source_multiple_runs_base:
    stats_multiple_runs[sname] = {}
    spath = source_multiple_runs_base_path.format(sdir)
    #   (flex, grid, price, missed) = read_data(spath, [0, 3, 5, 2])
    #   stats_ratio_impact[sname] = {'flex' : flex, 'grid' : grid, 'price': price, 'missed': missed}
    
    # columns : "Flexibility"	"Workload"	"Waiting Ratio"	"Missed Ratio"	"Grid Ratio"	"Total energy"	"Price"
    values = read_data(spath, [0, 1, 4, 5, 6, 3])
    zipped = zip(*values)
    
    # re-structure the data...
    #zip(price_factor, wl_instance, grid, energy, price, missed)
    result = []
    for flex in sorted(set(values[0])):
        good = [item for item in zipped if item[0] == flex]
        (_, wl_instance, grid, energy, price, missed) = zip(*good)
        
        stats_multiple_runs[sname][flex] = {'wli' : wl_instance, 'energy' : energy, 'grid' : grid, 'price': price, 'missed': missed}

In [ ]:
def statsf_grid_energy(algo, flex):
    global stats_multiple_runs
    stats = stats_multiple_runs[algo][flex]
    grid_energy = [stats['energy'][i] * stats['grid'][i] for i in range(len(stats['energy']))]
    return (np.mean(grid_energy), np.std(grid_energy))

def statsf_renew_energy(algo, flex):
    global stats_multiple_runs
    stats = stats_multiple_runs[algo][flex]
    grid_energy = [stats['energy'][i] * (1.0 - stats['grid'][i]) for i in range(len(stats['energy']))]
    return (np.mean(grid_energy), np.std(grid_energy))

def statsf_total_energy(algo, flex):
    global stats_multiple_runs
    stats = stats_multiple_runs[algo][flex]
    return (np.mean(stats['energy']), np.std(stats['energy']))

def statsf_grid_price(algo, flex):
    global stats_multiple_runs
    stats = stats_multiple_runs[algo][flex]
    return (np.mean(stats['price']), np.std(stats['price']))

def statsf_qos_missed(algo, flex):
    global stats_multiple_runs
    stats = stats_multiple_runs[algo][flex]
    return (np.mean(stats['missed']) * 100.0, np.std(stats['missed']) * 100.0)

def statsf_renew_increase(algo, flex):
    (base, _) = statsf_renew_energy('firstfit', flex)
    (comp, _) = statsf_renew_energy(algo, flex)
    return ((comp - base) / base * 100.0, None)

def statsf_renew_increase_2(algo, flex):
    global stats_multiple_runs
    stats = stats_multiple_runs[algo][flex]
    base_stats = stats_multiple_runs['firstfit'][flex]
    
    rel = []
    for i in range(len(stats)):
        a = stats['energy'][i] * (1.0 - stats['grid'][i])
        b = base_stats['energy'][i] * (1.0 - base_stats['grid'][i])
        rel.append((a - b) / float(b))
        
    return (np.mean(rel)* 100.0, None)

def statsf_cost_decrease(algo, flex):
    (base, _) = statsf_grid_price('firstfit', flex)
    (comp, _) = statsf_grid_price(algo, flex)
    return ((base - comp) / base * 100.0, None)

def statsf_grid_decrease(algo, flex):
    (base, _) = statsf_grid_energy('firstfit', flex)
    (comp, _) = statsf_grid_energy(algo, flex)
    return ((base - comp) / base * 100.0, None)

In [ ]:
def draw_chart_compare_multirun(stats_func, name, items, styles = None, title = None, ybounds=(None, None),
                         legend = False, xlabel=r"Workload flexibility factor ($flexf$)", ylabel=r"Ratio"):
    fig, ax1 = plt.subplots()
    
    flex_factors = [0, 1, 2, 4, 8, 16]
    
    if styles is None:
        global plot_compare_approaches
        styles = plot_compare_approaches
    
    stylef = lambda n: getpstyle(styles, n)
    
    if title is not None:
        ax1.set_title(title)
    ax1.set_ylabel(ylabel)
    ax1.set_xlabel(xlabel)

    for item in items:
        (yval, stderr) = zip(*[stats_func(item, f) for f in flex_factors])
        #(yval, stderr) = stats_func(item, flex_factors)
        ax1.plot(flex_factors, yval, **stylef(item))
    
    
    ax1.set_axisbelow(True)
    ax1.grid(zorder=0, axis='y')
    ax1.set_xticks(flex_factors)
    
    ax1.set_ybound(lower=ybounds[0], upper=ybounds[1])
    ax1.set_xbound(lower=min(flex_factors)-0.5, upper=max(flex_factors)+0.5)
    
    if legend :
        lgd = ax1.legend(loc="upper center", bbox_to_anchor=(0.5, 1.20), ncol=2)
        fig.savefig("output/flex_base/{}.pdf".format(name), bbox_extra_artists=(lgd,), bbox_inches='tight')
    else:
        fig.savefig("output/flex_base/{}.pdf".format(name))

In [ ]:
draw_chart_compare_multirun(statsf_grid_energy, "compare_grid_consumption_a", algos_set_noprice, ylabel="Grid energy consumed (kW.h)")

In [ ]:
draw_chart_compare_multirun(statsf_grid_energy, "compare_grid_consumption_b", algos_set_price, ylabel="Grid energy consumed (kW.h)")

In [ ]:
draw_chart_compare_multirun(statsf_grid_price, "compare_grid_cost_a", algos_set_noprice, ylabel="Grid energy consumed (kW.h)")

In [ ]:
draw_chart_compare_multirun(statsf_grid_price, "compare_grid_cost_a", algos_set_price, ylabel="Grid energy consumed (kW.h)")

In [ ]:
draw_chart_compare_multirun(statsf_grid_price, "compare_grid_price_a", algos_set_noprice, ylabel="Grid energy cost ($)")

In [ ]:
draw_chart_compare_multirun(statsf_grid_price, "compare_grid_price_b", algos_set_price, ylabel="Grid energy cost ($)")

In [ ]:
draw_chart_compare_multirun(statsf_qos_missed, "compare_fail_rate_a", algos_set_noprice,
                            legend=True, ylabel="SLA violation (%)")

In [ ]:
draw_chart_compare_multirun(statsf_qos_missed, "compare_fail_rate_b", algos_set_price,
                            legend=True, ylabel="SLA violation (%)")

### Comparison between greenslot variants

In [ ]:
draw_chart_compare_multirun(statsf_qos_missed, "compare_fail_rate_gs", algos_set_gs, ylabel="SLA violation (%)")
print(statsf_qos_missed('greenslot-original', 0))

In [ ]:
draw_chart_compare_multirun(statsf_qos_missed, "compare_fail_rate_gs_zoomed", 
                     algos_set_gs, ylabel="SLA violation (%)", legend=True, ybounds=(0, 3))

In [ ]:
draw_chart_compare_multirun(statsf_grid_energy, "compare_grid_consumption_gs", algos_set_gs, ylabel="Grid energy consumed (kW.h)")

In [ ]:
draw_chart_compare_multirun(statsf_grid_price, "compare_grid_price_gs", algos_set_gs, ylabel="Grid energy cost ($)")

In [ ]:
def draw_bar_compare_flex_1(statfunc, series, name, ylabel, title=None, legendloc='lower left', legend=False,
                            xlabel=r"Facteur de flexibilité de la charge", customize=None):
    
    fig, ax1 = plt.subplots()
    
    flex_factors = [2, 8, 16]
    labels = ["{}".format(x) for x in flex_factors]
    
    global bar_compare_approaches
    stylef = lambda n: getbstyle(bar_compare_approaches, n)

    if title is not None:
        ax1.set_title(title)
    ax1.set_ylabel(ylabel)
    ax1.set_xlabel(xlabel)

    width = 1.0 / (len(series) + 2)
    curdelta = 0.0
    indices = np.arange(len(flex_factors))


    for algo in series:
        vals = []
        stds = []
        use_stds = True
        for flex in flex_factors:
            (val, std) = statfunc(algo, flex)
            vals.append(val)
            stds.append(std)
            if std is None:
                use_stds = False
        if use_stds:        
            ax1.bar(indices + curdelta , vals, width, yerr=stds, zorder=1, **stylef(algo))
        else:
            ax1.bar(indices + curdelta , vals, width, zorder=1, **stylef(algo))
        curdelta += width


    ax1.set_xticks(indices + width * ((len(series)) / 2.0) )
    ax1.set_xticklabels(tuple(labels))
    ax1.set_xbound(-width, (len(flex_factors) - 1.0) + (len(series) + 1) * width)
    
    ax1.set_axisbelow(True)
    ax1.grid(zorder=0, axis='y')
    
    #if legendloc is not None:
    #    ax1.legend(loc=legendloc)
        
    if customize is not None:
        customize(ax1, fig)
        
    figpath = os.path.join("output/multiple_runs", name + ".pdf")
    if legend :
        lgd = ax1.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), ncol=1)
        fig.savefig(figpath, bbox_extra_artists=(lgd,), bbox_inches='tight')
    else:
        fig.savefig(figpath)

In [ ]:
draw_bar_compare_flex_1(statsf_grid_energy, algos_set_final, 'bar_grid_consumption',
                        ylabel='Énergie non renouvelable (kW.h)', legendloc='lower left', legend=False)

In [ ]:
draw_bar_compare_flex_1(statsf_total_energy, algos_set_final, 'bar_total_consumption',
                        ylabel='Total energy consumption (kW.h)', legendloc='lower left')

In [ ]:
draw_bar_compare_flex_1(statsf_grid_price, algos_set_final, 'bar_grid_price',
                        ylabel='Coût de l\'énergie achetée ($)', legendloc='lower left',
                        legend = True)

In [ ]:
draw_bar_compare_flex_1(statsf_qos_missed, algos_set_final, 'bar_deadline_failed',
                        ylabel='SLA violation (%)', legendloc='upper left', legend=False,
                        customize= lambda ax, fig : ax.set_ybound(lower=0.0))

### Synthetic metrics with first-fit comparison
Evaluation of the increase or decrease of several metrics (grid energy used, renewable energy used, grid energy cost).

In [ ]:
draw_bar_compare_flex_1(statsf_renew_increase, algos_set_final_noff, 'bar_renew_increase',
                        ylabel='Increase of renewable energy use (%)', legendloc='upper left')

In [ ]:
draw_bar_compare_flex_1(statsf_cost_decrease, algos_set_final_noff, 'bar_cost_saving',
                        ylabel='Cost saving (%)', legendloc='upper left')

In [ ]:
draw_bar_compare_flex_1(statsf_grid_decrease, algos_set_final_noff, 'bar_grid_decrease',
                        ylabel='Decrease of grid energy use (%)', legendloc='upper left')

# Some data tables...

In [ ]:
table_stats = {
    'grid': (statsf_grid_energy, 'Grid energy (kWh)', '{0:.1f}'),
    '%grid': (statsf_grid_decrease, 'Grid energy saving (\\%)', '{0:.1f}'),
    '%cost': (statsf_cost_decrease, 'Cost saving (\\%)', '{0:.1f}'),
    'sla': (statsf_qos_missed, 'SLA violation (\\%)', '{0:.2f}'),
}

In [ ]:
table_algos = {
    'wsum' : ('Weighted sum (A)', 'WS'),
    'wsinhsum' : ('Weighted sinh sum (A)', 'WSH'),
    'fuzzy-it' : ('Fuzzy-based IT (A)', 'FIT'),
    'price-wsum' : ('Weighted sum (B)', 'WS'),
    'price-wsinhsum' : ('Weighted sinh sum (B)', 'WSH'),
    'price-fuzzy-it' : ('Fuzzy-based IT (B)', 'FIT'),
    'greenslot-original' : ('GreenSlot (original)', 'Orig.'),
    'greenslot-improved' : ('GreenSlot (part modif)', 'P.Mod.'),
    'greenslot-full' : ('GreenSlot (modified)', 'Mod.'),
    'firstfit' : ('First-fit', 'FF'),
}

In [ ]:
def items2line(items):
    return (" & ").join(items) + r" \\"

In [ ]:
def print_partial_table_1(flex, stats, algos, ts = table_stats, ta = table_algos):    
    header = ["Approach", "Variant"]
    for metric in stats:
        header.append(ts[metric][1])

    print(items2line(header))

    for algo in algos:
        line = ["", ta[algo][0]]
        for metric in stats:
            (f, _, fmt) = ts[metric]
            val = f(algo, flex)[0]
            
            if fmt is not None:
                val = fmt.format(val)
            line.append(val)

        print(items2line(line))

In [ ]:
def print_partial_table_2(flex, stats, algos, ts = table_stats, ta = table_algos):    
    header = [""]
    for algo in algos:
        header.append(ta[algo][1])

    print(items2line(header))

    for metric in stats:
        line = [ts[metric][1]]
        for algo in algos:
            (f, _, fmt) = ts[metric]
            val = f(algo, flex)[0]
            
            if fmt is not None:
                val = fmt.format(val)
            line.append(val)

        print(items2line(line))

In [ ]:
algos_set_table = ['wsum', 'wsinhsum', 'fuzzy-it', 'price-wsum', 'price-wsinhsum', 'price-fuzzy-it', 'greenslot-original', 'greenslot-improved', 'greenslot-full']

In [ ]:
print_partial_table_1(2, ['%grid', '%cost', 'sla'], algos_set_final)

In [ ]:
print_partial_table_2(16, ['%grid', '%cost', 'sla'], algos_set_table)

In [ ]:
print_partial_table_2(2, ['%grid', '%cost', 'sla'], algos_set_table)

## Some values used in text


In [ ]:
f = 8
for algo in ['wsum', 'wsinhsum', 'fuzzy-it']:
    (grida, _) = statsf_grid_energy(algo, f)
    (gridb, _) = statsf_grid_energy("price-{}".format(algo), f)
    gridinc = (gridb-grida)/grida * 100.0
    
    (costa, _) = statsf_grid_price(algo, f)
    (costb, _) = statsf_grid_price("price-{}".format(algo), f)
    costdec = (costa-costb)/costa * 100.0
    
    print("Using variant B with {0} : increase energy by {1:.2f}%, decrease price by {2:.2f}%.".format(algo, gridinc, costdec))
    

In [ ]:
algo = 'price-fuzzy-it'
for f in [2, 8, 16]:
    (grida, _) = statsf_grid_energy(algo, f)
    (gridb, _) = statsf_grid_energy("greenslot-full", f)
    griddec = (gridb-grida)/gridb * 100.0
    
    (costa, _) = statsf_grid_price(algo, f)
    (costb, _) = statsf_grid_price("greenslot-full", f)
    costdec = (costb-costa)/costb * 100.0
    
    print("Using {0} vs GreenSlot : decrease energy by {1:.2f}%, decrease price by {2:.2f}%.".
          format(algo, griddec, costdec))

In [ ]:
algo = 'price-wsum'
for f in [2, 8, 16]:
    (grida, _) = statsf_grid_energy(algo, f)
    (gridb, _) = statsf_grid_energy("greenslot-full", f)
    griddec = (gridb-grida)/gridb * 100.0
    
    (costa, _) = statsf_grid_price(algo, f)
    (costb, _) = statsf_grid_price("greenslot-full", f)
    costdec = (costb-costa)/costb * 100.0
    
    print("Using {0} vs GreenSlot : decrease energy by {1:.2f}%, decrease price by {2:.2f}%.".
          format(algo, griddec, costdec))

Show, for each flex value, the difference between our best heuristic and GreenSlot modified.

In [ ]:
flexf = [0, 1, 2, 4, 8, 16]
algos = [a for a in table_algos if not a.startswith('greenslot')]
#algos = ['price-fuzzy-it']
refalgo = 'greenslot-full'

for metric in [('Grid energy', statsf_grid_energy, lambda a, b : a < b), ('Energy cost', statsf_grid_price, lambda a, b: a < b)]:
    print("Comparison of approaches for metric '{}':".format(metric[0]))
    for f in flexf:
        (ref, _) = metric[1](refalgo, f)
        best = None
    
        for a in algos:
            (cur, _) = metric[1](a, f)
            if best is None or metric[2](cur, best[0]):
                best = (cur, a)
        
        gain = float(ref - best[0]) * 100.0 / ref
        print(" [flexf = {0}] best is {1} ({2:+.2f}%)".format(f, best[1], gain))
        

### Repartition of priority groups accross tasks
Not found any simple way to get the exact values (truncated expon distribution, not so common).
Empirical values should be enough. (*?*)

In [ ]:
import scipy.stats as ss

def truncated_expon(l):
    while True:
        ret = ss.expon.rvs(scale = 1.0/l)
        if ret <= 1.0:
            return ret

def get_gprio(p):
    return int(p*3.0)
    

In [ ]:
counts = [0] * 4
for i in range(10**6):
    counts[get_gprio(truncated_expon(6))] += 1
#counts = [vals.count(i) for i in range(4)]
counts

In [ ]:
[float(c) * 100.0 / sum(counts) for c in counts]